### Besides the analysis in the main notebook, we also did an experiment in text mining. Since there are limits in accessing tweets, there are not enough text data, so we only use it as an additional trial here instead of including it in the main line.

### Use Twitter API to access the text data about players
We use Twitter API to access the text data, and select netizens’ comments on NBA active players and scrape them as texts. We collect approximately 80000 tweets about 540 players, which shows their performance this NBA season, and export them as uniform form for every player

In [ ]:
consumer_key = "yujgy2FPd9IOcQsR7Y0co19cI"
consumer_secret = "OPZdiMIZzA7Jm00Z7PoNIQOfuqdC1T9XMHzk4BDS6tRE8qcLEE"
access_token = "1183568791772504066-galXwo5tkeGIhcfRo9HkdHfTdIl5od"
access_token_secret = "Kba56HaU3BGVRDd6Jr7EUjcNzTQ5Ag9dzMUt2Y0B1nNOh"
# you can use these passwords to examine the correctness of our codes

In [2]:
import tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [3]:
import pandas as pd
df = pd.DataFrame(columns = ['Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date'])

In [4]:
df_players = pd.read_excel('player18.xlsx')

Get the name data for players in Season 2018-2019

In [5]:
players = df_players['Player'] 
len(players) 

540

In [6]:
import datetime as dd
df1 = pd.DataFrame(columns = ['date','Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date'])

Original way of using API to get the data in a single document, the original code manages to return all the data with tweepy.Cursor, while it will report an error 429 as follows because of data getting limit of Twiiter API

'''The error is just for showing the traditional low-efficient way and we improve it'''

In [12]:
write = pd.ExcelWriter('all_pla_cri.xlsx')
for player in players2:
    df1 = pd.DataFrame(columns = ['date','Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date'])
    search_term = player
    i = 0
    i_lst = [0,]
    for j in range(21,29): # because of the 7-day constraint of twitter API data, we can only get the data from Nov 21th to Nov 28th on Nov 28th
        for tweet in tweepy.Cursor(api.search, q=search_term, count=100, since="2019-11-"+str(j), until="2019-11-"+str(j+1), lang='en').items():
            print(i, end='\r')
            df1.loc[i,'date'] = tweet.created_at.strftime('%Y-%m-%d')
            df1.loc[i, 'Tweets'] = tweet.text
            df1.loc[i, 'User'] = tweet.user.name
            df1.loc[i, 'User_statuses_count'] = tweet.user.statuses_count        
            df1.loc[i, 'user_followers'] = tweet.user.followers_count
            df1.loc[i, 'User_location'] = tweet.user.location
            df1.loc[i, 'User_verified'] = tweet.user.verified
            df1.loc[i, 'fav_count'] = tweet.favorite_count
            df1.loc[i, 'rt_count'] = tweet.retweet_count
            df1.loc[i, 'tweet_date'] = tweet.created_at
            i+=1
            if i == 1500+i_lst[j-21]: #if there are more than 1500 tweets for a player, return only the first 1500 of them.
                break
        i_lst.append(i)
    df1.to_excel(write,sheet_name='{}'.format(str(player)),index=False) # use the names of the player as names of the sheet
write.save()    # the original code manages to return all the data with tweepy.Cursor, while it will report an error 429 because of data getting limit of Twiiter API

TweepError: Twitter error response: status code = 429

In [14]:
import time

The actual implemented way of getting the data 20 players at a time. Run this code will take a few days totally. The final results are in the zip file

<b>'''As for the large dataset, Twitter API will take a long time to run'''

In [18]:
for i in range(0,540,20):
    players2 = players[i:i+20] # get the data of 20 players at a time
    write = pd.ExcelWriter('all_pla_cri{}.xlsx'.format(str(i))) #create a xlsx file for every 20 players in the player list
    for player in players2:
        df1 = pd.DataFrame(columns = ['date','Tweets', 'User', 'User_statuses_count', 
                             'user_followers', 'User_location', 'User_verified',
                             'fav_count', 'rt_count', 'tweet_date'])
        search_term = player
        i = 0
        i_lst = [0,]
        for j in range(21,29):
            for tweet in tweepy.Cursor(api.search, q=search_term, count=100, since="2019-11-"+str(j), until="2019-11-"+str(j+1), lang='en').items():
                print(i, end='\r')
                df1.loc[i,'date'] = tweet.created_at.strftime('%Y-%m-%d')
                df1.loc[i, 'Tweets'] = tweet.text
                df1.loc[i, 'User'] = tweet.user.name
                df1.loc[i, 'User_statuses_count'] = tweet.user.statuses_count        
                df1.loc[i, 'user_followers'] = tweet.user.followers_count
                df1.loc[i, 'User_location'] = tweet.user.location
                df1.loc[i, 'User_verified'] = tweet.user.verified
                df1.loc[i, 'fav_count'] = tweet.favorite_count
                df1.loc[i, 'rt_count'] = tweet.retweet_count
                df1.loc[i, 'tweet_date'] = tweet.created_at
                i+=1
                if i == 1500+i_lst[j-21]:
                    break
            i_lst.append(i)
        df1.to_excel(write,sheet_name='{}'.format(str(player)),index=False)
    write.save()
    time.sleep(600)  # use sleep(600) so that we can run the codes without encountering 429 error
    # this is the code we finally used to get the data and stored the data for every 20 players into a file

### Abstract text data

Form a list in which each element represents the name of an output document

In [ ]:
import pandas as pd
lst=[i for i in range(0,540,20)] 

In [ ]:
df_players = pd.read_excel('player18.xlsx')
players = list(df_players['Player'])

<b>'''As for the large dataset, this type transformation process will take a long time to run'''

In [ ]:
for i,player in enumerate(players):
    for j in range(len(lst)-1):
        if i<lst[j+1] and i>=lst[j]:
            playerfr = pd.read_excel('all_pla_cri'+str(lst[j])+'.xlsx',sheet_name=player)
            f = open("{}.txt".format(player),"w+",encoding='utf-8')
            for k in range(len(playerfr)):
                f.write(playerfr["Tweets"][k])
            f.close() # write the data from the sheets of output files into text files

Output the tweet text files used for analyzing

In [ ]:
import nltk
from nltk.corpus import PlaintextCorpusReader
import pandas as pd
df_players = pd.read_excel('player18.xlsx')
players = list(df_players['Player'])
excel_code="text_doc/"
tweets_root = excel_code
player_lst=[]

<b>'''As for the large dataset, this type transformation process will take a long time to run'''

In [ ]:
for i,player in enumerate(players):  
    for j in range(len(lst)-1):
        if i<lst[j+1] and i>=lst[j]:
            playerfr = pd.read_excel(excel_code+'all_pla_cri'+str(lst[j])+'.xlsx',sheet_name=player)
            f = open("{}.txt".format(player),"w+",encoding='utf-8')
            for k in range(len(playerfr)):
                f.write(playerfr["Tweets"][k])
            f.close()

### Use nrc data on sentiment analysis

We do data cleaning by the following 2 rules: 
First, eliminate the players with scarce text (only have a few comments on Twitter ) 
Second, eliminate duplicate text in each player’s corpus ( same contents below the same player)

After cleaning, we build the data arrays used for comparative_emotion_analyzer，divide words into two major categories (Positive & Negative) and 8 different sentiment types (Surprise, Anticipation, Trust, Joy, Disgust, Anger, Sadness and Fear)

In [2]:
for i,player in enumerate(players): 
    player_file = "{}.*".format(player)
    player_data = PlaintextCorpusReader(tweets_root,player_file)
    try:
        player_lst.append([player,player_data.raw()])
    except:
        continue

<b>'''As for the large dataset, this sentiment analysis will take a long time to run'''

In [3]:
def get_nrc_data():   # the get_nrc_data/emotion_analyzer/comparative_emotion_analyzer function used in class
    nrc = "NRC-emotion-lexicon-wordlevel-alphabetized-v0.92.txt"
    count=0
    emotion_dict=dict()
    with open(nrc,'r') as f:
        all_lines = list()
        for line in f:
            if count < 46:
                count+=1
                continue
            line = line.strip().split('\t')
            if int(line[2]) == 1:
                if emotion_dict.get(line[0]):
                    emotion_dict[line[0]].append(line[1])
                else:
                    emotion_dict[line[0]] = [line[1]]
    return emotion_dict


def emotion_analyzer(text,emotion_dict=get_nrc_data()):
    #Set up the result dictionary
    emotions = {x for y in emotion_dict.values() for x in y}
    emotion_count = dict()
    for emotion in emotions:
        emotion_count[emotion] = 0

    #Analyze the text and normalize by total number of words
    total_words = len(text.split())
    for word in text.split():
        if emotion_dict.get(word):
            for emotion in emotion_dict.get(word):
                emotion_count[emotion] += 1/len(text.split())
    return emotion_count

def comparative_emotion_analyzer(text_tuples,object_name,print_output=False):
    if print_output:
        print("%-20s %1s\t%1s %1s %1s %1s   %1s %1s %1s %1s"%(object_name,
                                                              "negative","positive","surprise",
                                                              "anticipation","trust","joy","disgust","anger","sadness","fear"))
                                                              
    import pandas as pd
    df = pd.DataFrame(columns=[object_name,'Negative',
                           'Positive','Surprise','Anticipation',
                           'Trust',"Joy","Disgust","Anger","Sadness","Fear"])
    df.set_index(object_name,inplace=True)
    
    output = df
    for text_tuple in text_tuples:
        text = text_tuple[1] 
        result = emotion_analyzer(text)
        if print_output:
            print("%-20s %1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f\t%1.2f"%(
                text_tuple[1][0:20],result['negative'],result['positive'],result['surprise'],
                  result['anticipation'],result['trust'],result['joy'],result['disgust'],result['anger'],result['sadness'],result['fear']))
        df.loc[text_tuple[0]] = [result['negative'],result['positive'],result['surprise'],
                  result['anticipation'],result['trust'],result['joy'],result['disgust'],result['anger'],result['sadness'],result['fear']]
    return output

In [4]:
dff=comparative_emotion_analyzer(player_lst,object_name='player',print_output=False) #run the built function to get 

In [5]:
dff.to_excel('emo_player-all.xlsx') #output the analyzing result to excel

### Build Machine Learning models

In [3]:
df1=pd.read_csv('Final data.csv',index_col='Unnamed: 0')
df2=pd.read_excel('emo_player-all.xlsx')

In [4]:
df1=df1[1:]
df1=df1[df1['Year']==2018]

In [5]:
df1['Year'].unique()

array([2018], dtype=int64)

In [6]:
df=pd.merge(df1,df2,left_on=["Name"],right_on=["player"],how="inner")

In [7]:
df.columns
df=df[['Name', 'tenure', 'AGE',
       'GP', 'W', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'PF', 'FP', 'DD2', '+/-', 'Height', 'Weight',
       'Salary_Weight', 'Salary_Weight_l',
       'Negative', 'Positive', 'Positive/Negative', 'Surprise', 'Anticipation',
       'Trust', 'Joy', 'Disgust', 'Anger', 'Sadness', 'Fear']]
x_train=df[['tenure', 'AGE',
       'GP', 'W', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'PF', 'FP', 'DD2', '+/-', 'Height', 'Weight',
        'Salary_Weight_l',
       'Negative', 'Positive', 'Positive/Negative', 'Surprise', 'Anticipation',
       'Trust', 'Joy', 'Disgust', 'Anger', 'Sadness', 'Fear']]
y_train=df['Salary_Weight']
x_train2=df[['tenure', 'AGE',
       'GP', 'W', 'MIN', 'PTS', 'FGM', 'FGA', 'FG%', '3PA', '3P%',
       'FTM', 'FTA', 'FT%', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'PF', 'FP', 'DD2', '+/-', 'Height', 'Weight',
        'Salary_Weight_l']]      # the names of the columns

Instead of using our original best model Random Forest under Auto-Regression Assumption, we decided to apply bagging method under Auto-Regression Assumption to the the new data set with sentiment features due to a limit in the data size.


In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingRegressor
import math

parameters = {
     'n_estimators':[500], #the number of trees
     'max_samples':(0.5, 0.7, 0.8),
     'max_features':[20],
     'random_state':[888]
     #'oob_score':np.ravel(y_test)
     
}

model = GridSearchCV(BaggingRegressor(),parameters,cv=10,iid=False,scoring='neg_mean_squared_error')
model.fit(x_train, y_train)
math.sqrt(-model.best_score_), model.best_params_  # best model parameters are n_estimators: 500, and max_features: 20

(0.06561011360407598,
 {'max_features': 20,
  'max_samples': 0.5,
  'n_estimators': 500,
  'random_state': 888})

In [11]:
parameters2 = {
     'n_estimators':[500], #the number of trees
     'max_samples':(0.5, 0.7, 0.8),
     'max_features':[20],
     'random_state':[888]
     #'oob_score':np.ravel(y_test)
     
}

model = GridSearchCV(BaggingRegressor(),parameters2,cv=10,iid=False,scoring='neg_mean_squared_error')
model.fit(x_train2, y_train)
math.sqrt(-model.best_score_), model.best_params_ # this model includes sentiment attributes

(0.06496773952486219,
 {'max_features': 20,
  'max_samples': 0.7,
  'n_estimators': 500,
  'random_state': 888})